## Workflow 
##### (especificado no Canvas pelo prof. Leandro)

- Baixar os dados do MovieLens 100k (uma base contendo informações sobre filmes assistidos e avaliados por vários usuários)
- Rodar o algoritmo de filtragem colaborativa baseada no usuário (basicamente o algoritmo KNN) nesses dados. Para isso você pode usar a biblioteca [surpriselib](http://surpriselib.com/). [Aqui](http://surprise.readthedocs.io/en/stable/knn_inspired.html) você tem os algoritmos baseados em KNN que pode usar.
- O que devo entregar?
    - Link para código comentado no github.
    - Qualidade do modelo em termos de RMSE (veja seção Getting Started na página [http://surpriselib.com/](http://surpriselib.com/))
    - Uma demo web onde você passar um usuário, mostra as top-5 recomendaçõe e apresenta os três melhores vizinhos desse usuário)


## Solução

Inicialmente, vou importar as bibliotecas necessárias para implementar o código, entre elas encontra-se a biblioteca **surpriselib** mencionada acima.

In [2]:
from surprise import Dataset, evaluate
from surprise import KNNBasic
from surprise.model_selection import cross_validate
from collections import defaultdict
import os, io

#### Constantes

In [10]:
QUANTIDADE_DE_RECOMENDACOES = 5
QUANTIDADE_DE_VIZINHOS_DO_USUARIO = 3

#### Métodos Auxiliares

In [8]:
def get_top_recommendations(predictions, N = QUANTIDADE_DE_RECOMENDACOES):
    """ Este método é responsável por retornar as top N recomendações de filmes para todos os usuários baseado
    nos cálculos de predições realizados. O retorn deste método é uma lista de IDs."""
    
    top_recs = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_recs[uid].append((iid, est))
    
    for uid, user_ratings in top_recs.items():
        user_ratings.sort(key = lambda x: x[1], reverse = True)
        top_recs[uid] = user_ratings[:N]
    
    return top_recs

In [9]:
def convert_item_ids_to_names():
    """Este método converte um ID do filme para o título do filme. Visto que o método que retorna as top N recomendações
    retorna uma lista de IDs de filmes, este método se faz necessário."""
    file_name = (os.path.expanduser('~') + '/.surprise_data/ml-100k/ml-100k/u.item')
    rid_to_name = {}
    with io.open(file_name, 'r', encoding='ISO-8859-1') as f:
        for line in f:
            line = line.split('|')
            rid_to_name[line[0]] = line[1]
    return rid_to_name

#### Workflow de Solução

In [15]:
# Load do MovieLens 100K Dataset (https://grouplens.org/datasets/movielens/100k/)
data = Dataset.load_builtin("ml-100k")

# Cria o training set
training_set = data.build_full_trainset()

# Visto que trata-se de collaborative filtering baseado em usuário, seto o user_based com True
sim_options = {
  'name': 'pearson_baseline',
  'user_based': True
}

# Algoritmo utilizado é o k-NN
knn = KNNBasic(sim_options=sim_options)

# Treina o training set
knn.fit(training_set)

# Cria o test set
test_set = training_set.build_anti_testset()

predictions = knn.test(test_set)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [16]:
# 5 top recomendações de todos os usuários
top_recommendations = get_top_recommendations(predictions)

In [17]:
# converte IDs de filmes em títulos de filmes
ids_to_names = convert_item_ids_to_names()

In [ ]:
# Entrada de ID de usuário via linha de comando
uid = str(input("Digite um ID: "))